<a href="https://colab.research.google.com/github/danielbehargithub/LinkedIn_Salary/blob/main/Profile_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playwright
!playwright install


In [70]:
# Enter url, email and password
user_url = "https://www.linkedin.com/in/daniel-behar-168647280"
email = "daniel10behar@gmail.com"
password = "alivaba1"


In [72]:
import nest_asyncio
from playwright.async_api import async_playwright
import asyncio

async def scrape_user_profile(user_url, email, password):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # Running in headless mode
        page = await browser.new_page()

        print("Navigating to LinkedIn login page...")
        # Navigate to LinkedIn login page
        await page.goto("https://www.linkedin.com/login")
        await asyncio.sleep(3)

        # Perform login
        print("Logging into LinkedIn...")
        await page.fill('input[id="username"]', email)
        await page.fill('input[id="password"]', password)
        await page.click('button[type="submit"]')
        await asyncio.sleep(5)

        # Check for two-step verification
        if await page.is_visible('input[name="pin"]'):
            print("Two-step verification detected. Please enter the code sent to your email.")
            verification_code = input("Enter the verification code: ")
            await page.fill('input[name="pin"]', verification_code)
            await page.click('button[type="submit"]')
            await asyncio.sleep(3)

        # Relevant sections to scrape for the user
        profile_sections = [
            "/details/education/",
            "/details/skills/",
            "/details/experience/",
            # "/details/certifications/"
        ]

        scraped_data = {}  # Dictionary to store HTML content of each section

        # Loop through the list of profile URLs
        for section in profile_sections:
            url = f"{user_url}{section}"
            try:
                print(f"Navigating to section: {url}")
                # Navigate to the section URL
                await page.goto(url)
                await asyncio.sleep(3)

                # Save the HTML content of the section page
                html = await page.content()
                section_name = section.strip("/").split("/")[-1]  # Keep only the part after the last slash
                scraped_data[section_name] = html  # Store HTML content in dictionary
                print(f"HTML content for {section_name} saved in memory.")

                # For debug
                filename = f"{section_name}.html"  # Ensure .html extension
                with open(filename, "w", encoding="utf-8") as f:
                    f.write(html)
                print(f"Saved HTML for {url} as {section_name}")

                # Optional: Take a screenshot of the section page
                screenshot_filename = f"{section_name}.png"
                await page.screenshot(path=screenshot_filename, full_page=True)
                print(f"Saved screenshot for {url} as {screenshot_filename}")

            except Exception as e:
                print(f"Failed to scrape {url}: {e}")

        # Close the browser
        print("Closing the browser...")
        await browser.close()

        return scraped_data  # Return all scraped HTML as a dictionary



In [73]:
scraped_html_data = await scrape_user_profile(user_url, email, password)

from bs4 import BeautifulSoup

# Example HTML from the dictionary
html_skills = scraped_html_data.get("skills")
html_education = scraped_html_data.get("education")
html_experience = scraped_html_data.get("experience")

# Parse HTML
soup_skill = BeautifulSoup(html_skills, "html.parser")
soup_education = BeautifulSoup(html_education, "html.parser")
soup_experience = BeautifulSoup(html_experience, "html.parser")


Navigating to LinkedIn login page...
Logging into LinkedIn...
Navigating to section: https://www.linkedin.com/in/daniel-behar-168647280/details/education/
HTML content for education saved in memory.
Saved HTML for https://www.linkedin.com/in/daniel-behar-168647280/details/education/ as education
Saved screenshot for https://www.linkedin.com/in/daniel-behar-168647280/details/education/ as education.png
Navigating to section: https://www.linkedin.com/in/daniel-behar-168647280/details/skills/
HTML content for skills saved in memory.
Saved HTML for https://www.linkedin.com/in/daniel-behar-168647280/details/skills/ as skills
Saved screenshot for https://www.linkedin.com/in/daniel-behar-168647280/details/skills/ as skills.png
Navigating to section: https://www.linkedin.com/in/daniel-behar-168647280/details/experience/
HTML content for experience saved in memory.
Saved HTML for https://www.linkedin.com/in/daniel-behar-168647280/details/experience/ as experience
Saved screenshot for https://ww

In [74]:
company_name_container = soup_experience.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold")
company_name = (
    company_name_container.find("span", {"aria-hidden": "true"}).get_text(strip=True)
    if company_name_container else "N/A"
)

duration_container = soup_experience.find("span", class_="t-14 t-normal")
if duration_container:
    duration_text = duration_container.find("span", {"aria-hidden": "true"}).get_text(strip=True)
    if "·" in duration_text:
        job_type, job_duration = [part.strip() for part in duration_text.split("·", 1)]
    else:
        job_type, job_duration = "N/A", duration_text
else:
    job_type, job_duration = "N/A", "N/A"

# הצגת התוצאות
print(f"Company Name: {company_name}")
print(f"Job Duration: {job_duration}")
print(f"job Type: {job_type}")


Company Name: Israel Defense Forces
Job Duration: 4 yrs
job Type: N/A


In [75]:
education_section = soup_education.find("main", {"aria-label": "Education"})

education_records = education_section.find_all("div", {"data-view-name": "profile-component-entity"})


for record in education_records:
    print("Education record:")

    # Institution name
    institution_container = record.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold")
    institution_name = institution_container.get_text(strip=True) if institution_container else "N/A"
    print(f"  Institution: {institution_name}")

    # Date range
    date_container = record.find("span", class_="pvs-entity__caption-wrapper")
    date_range = date_container.get_text(strip=True) if date_container else "N/A"
    print(f"  Date: {date_range}")

    # Additional description
    description_container = record.find("span", class_="t-14 t-normal")
    description_text = description_container.get_text(strip=True) if description_container else "N/A"
    print(f"  Description: {description_text}")

    # Skills
    skills_container = record.find("div", class_="display-flex align-items-center t-14 t-normal t-black")
    skills = []
    if skills_container:
        skills_span = skills_container.find("span", {"aria-hidden": "true"})
        if skills_span:
            skills_text = skills_span.get_text(strip=True)
            if "Skills:" in skills_text:
                skills_text = skills_text.replace("Skills:", "").strip()
            skills = [skill.strip() for skill in skills_text.split("·")]
    print(f"  Skills: {', '.join(skills)}")

    # Additional text
    additional_text_container = record.find("div", class_="inline-show-more-text--is-collapsed")
    additional_text = ""
    if additional_text_container:
        additional_span = additional_text_container.find("span", {"aria-hidden": "true"})
        if additional_span:
            additional_text = additional_span.get_text(" ", strip=True)
    print(f"  Additional Text: {additional_text}")


Education record:
  Institution: Technion - Israel Institute of TechnologyTechnion - Israel Institute of Technology
  Date: Oct 2020
  Description: N/A
  Skills: Statistical Data Analysis, Apache Spark, Java, Data Structures, PyTorch, Deep Learning, Django
  Additional Text: 


In [76]:
# Find all <a> tags with data-field="skill_page_skill_topic"
skill_links = soup_skill.find_all("a", {"data-field": "skill_page_skill_topic"})

# Extract skills
skills = []
for link in skill_links:
    # Find the <span> with aria-hidden="true" inside the link
    skill_span = link.find("span", {"aria-hidden": "true"})
    if skill_span:
        skill_text = skill_span.get_text(strip=True)
        skills.append(skill_text)

unique_skills = sorted(set(skills))
print("Unique skills:", unique_skills)


Unique skills: ['Apache Spark', 'Data Analysis', 'Data Structures', 'Deep Learning', 'Django', 'HTML', 'Java', 'PyTorch', 'Python (Programming Language)', 'SQL', 'Statistical Data Analysis']
